<a href="https://colab.research.google.com/github/javiersrf/previsao_demanda_tcc/blob/main/Codigo_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importacoes

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import TimeSeriesSplit

## Definicao de funcoes uteis

In [ ]:
def one_hot_encode(df, column):
  # One Hot Encoded na coluna
  one_hot_df = pd.get_dummies(df[column], prefix=column)
  df_encoded = pd.concat([df, one_hot_df], axis=1)
  df_encoded = df_encoded.drop(columns=column)
  return df_encoded

def parse_dataframe(data):
  copy_data = data.copy()
  copy_data["data_saida"] = pd.to_datetime(copy_data["data_saida"])
  copy_data = copy_data[copy_data["id_cliente"].notnull()]
  copy_data = copy_data.drop(copy_data[copy_data["preco_saida"] == 0].index)
  copy_data = copy_data.drop(copy_data[copy_data["preco_total"] == 0].index)
  copy_data["data_saida"] = pd.to_datetime(copy_data["data_saida"])
  copy_data["year"] = copy_data["data_saida"].dt.year
  copy_data["month"] = copy_data["data_saida"].dt.month
  uniques_ids_grouped_by_month = copy_data.groupby(['year', 'month', 'id_produto_saida', ])['id_cliente'].nunique().reset_index()

  df_merged = pd.merge(copy_data, uniques_ids_grouped_by_month, on=['year', 'month', 'id_produto_saida'])
  df_merged = df_merged.rename(columns={
      "id_cliente_x":"id_cliente",
      "id_cliente_y":"quant_clientes_mes"
  })
  # Agrupando por mes
  df_grouped  = df_merged.groupby(['year', 'month', 'id_produto_saida', "quant_clientes_mes"]).agg({
    "preco_saida":"mean",
    "quantidade_saida":"sum",
    "preco_total":"sum",
  }).reset_index()
  final_data = one_hot_encode(df_grouped, "id_produto_saida")
  return final_data

In [ ]:
def create_date_from_month_year(df):
  df['date'] = pd.to_datetime(df[['year', 'month']].assign(Day=1)).dt.date
  df = df.sort_values("date")
  return df
data = create_date_from_month(data)

In [ ]:
class InputData:
  def __init__(self, year, month, product_id, clients_amount, total_price, price) -> None:
    self.year = year
    self.month = month
    self.product_id = product_id
    self.clients_amount = clients_amount
    self.total_price = total_price
    self.price = price

  def encode_product(self, product_id):
    products_columns = {column: 0 for column in data.columns if "id_produto_saida_" in column}
    products_columns["id_produto_saida_"+str(product_id)] = 1
    return [item for key, item in products_columns.items()]

  @property
  def to_input(self):
    return np.array([
        self.year,
        self.month,
        self.clients_amount,
        self.price,
        self.total_price,
        *self.encode_product(self.product_id)
    ])

## Carregamento Global

**Carregando arquivo**

In [ ]:
data = pd.read_csv("/content/demand_by_month.xlsx").drop(columns="Unnamed: 0")

**Dividindo os dados de teste e treino a partir da data selecionada**

In [ ]:
divisor =  "2022-01-01"
train = index_data[index_data["data"] < split_date]
train.to_excel("train_data.xlsx")
test = index_data[index_data["data"] >= split_date]
test.to_excel("test_data.xlsx")

## Machine Learning

**Parseando campo de data para formato datetime**

In [ ]:
index_data = data.copy()
index_data["data"] =  data["month"].astype(str) + "-1-"+ data["year"].astype(str)
index_data["data"] = pd.to_datetime(index_data["data"])

**Treinando modelos**

In [ ]:
target = "quantidade_saida"
predicts = {}
scores_ = []
seed = 7
_models = {}
_models["KNR"] = KNeighborsRegressor()
_models["LR"] = LinearRegression()
_models["RG"] = Ridge()
_models["LSVR"] = LinearSVR()
_models["DTR"] = DecisionTreeRegressor()
_models["RFR"] = RandomForestRegressor()
_models["GBR"] = GradientBoostingRegressor()
results = []
names = []
scoring = 'accuracy'
for name, model in _models.items():
  X_train, X_test = train.drop([target, "data"], axis=1).values, test.drop([target, "data"], axis=1).values
  y_train, y_test = train[target].values, test[target].values
  # evaluate each model in turn
  results = []
  names = []
  scoring = 'accuracy'
  model.fit(X_train, y_train)
  score = model.score(X_test, y_test)*100
  scores_.append((name, score))
  msg = "%s: %f %%" % (name, score.mean())
  print(msg)
  predicts[name] = model.predict(X_test)


**Validando resultado**

In [ ]:
y_predict = _models["GBR"].predict(X_test)
plt.plot(y_test.values)
plt.plot(y_predict)
plt.show()

In [ ]:
sns.scatterplot(x=y_predict, y=y_test.values)
plt.show()

**Salvando resultados em um arquivo**

In [ ]:
for name, values in predicts.items():
  test[name] = values
test.to_excel("results_ml.xlsx")

## Metodos Estatisticos

**Copiando dados para uma nova variavel e revertento o "one hot encoding"**

In [ ]:
statistic_data = data.copy()
dummies_columns = [col for col in statistic_data.columns if "id_produto_saida_" in col]
def find_product_id(df):
  for col in dummies_columns:
    if df[col] == 1:
      return int(col[17:])
data["id_produto_saida"] = statistic_data.apply(find_product_id, axis=1)
reverted_data = statistic_data.drop(columns=dummies_columns)

**Definindo apenas os produtos de categoria A como estudo**

In [ ]:
ONLY_A_PRODUCTS = True
if ONLY_A_PRODUCTS:
  PRODUCTS_IDS = [132,148,129,133,108,100,130,134,98,131,147]
else:
  PRODUCTS_IDS = list(reverted_data["id_produto_saida"].unique())

**Analise de demanda total pelo tempo**

In [ ]:
reverted_data[reverted_data["id_produto_saida"].isin(PRODUCTS_IDS)].groupby(["year", "month"]).agg({"preco_total":"sum"}).plot()

**Prevendo usando modelos estatisticos**

In [ ]:
results_values  = []
for product_id in [PRODUCTS_IDS]:
  data_product = reverted_data[reverted_data["id_produto_saida"] == product_id][["id_produto_saida", "quantidade_saida", "date", "year", "month"]]
  monthly_data_product = data_product.groupby(["year", "month", "id_produto_saida"]).agg({"quantidade_saida":"sum"}).reset_index()
  monthly_data_product['date'] = pd.to_datetime(monthly_data_product[['year', 'month']].assign(Day=1)).dt.date
  monthly_data_product = monthly_data_product.sort_values("date")
  monthly_data_product.head(5)
  #media movel simples
  for window in [3,5,7]:
    monthly_data_product[f'moving_avg_{window}'] = monthly_data_product['quantidade_saida'].rolling(window=window).mean()


  ##media movel ponderada
  pesos_n = {
        2: [0.3, 0.7],
        3: [0.2, 0.3, 0.5],
        4: [0.1, 0.2, 0.3, 0.4]
    }
  for key, value in pesos_n.items():
    monthly_data_product[f'weighted_moving_avg_{key}'] = monthly_data_product['quantidade_saida'].rolling(window=key).apply(lambda x: np.dot(x, value), raw=True)


  #ARIMA
  indexed_data = monthly_data_product.copy()
  indexed_data["date"] = pd.to_datetime(indexed_data["date"])
  indexed_data = indexed_data.set_index("date")
  train = indexed_data[indexed_data.index < split_date]
  test = indexed_data[indexed_data.index >= split_date]
  model = None
  model = auto_arima(train["quantidade_saida"], seasonal=True, m=12)
  forecasts = model.predict(test.shape[0])
  temp = np.empty(len(train))
  temp[:] = np.nan
  temp = np.concatenate(( temp, forecasts.values))
  monthly_data_product["ARIMA"] = temp
  results_values.append(monthly_data_product.copy())



**Concatenar todos os resultados**

In [ ]:
concact_results = pd.concat(results_values)
estastitical_data = pd.merge(reverted_data, concact_results, how="left", on=["year", "month", "id_produto_saida"])

## Juntando resultados de M.l com estatisticos

**Limpando os valores sem um dos resultados**

In [ ]:
clean_final_data = estastitical_data.dropna()


In [ ]:
ml_results = test.copy()
ml_results["id_produto_saida"] = ml_results.apply(find_product_id, axis=1)
n_ml_results = ml_results.drop(columns=dummies_columns)
final = pd.merge(clean_final_data, n_ml_results, on=["year", "month", "quant_clientes_mes", "id_produto_saida"], how="left")
final = final.drop(columns=[col for col in final.columns if "_x" in col or "_y" in col])
final.tail(4)
real_value = 'quantidade_saida'
columns_with_results = [
    'moving_avg_3', 'moving_avg_5', 'moving_avg_7', 'weighted_moving_avg_2',
    'weighted_moving_avg_3', 'weighted_moving_avg_4', 'ARIMA',
    'KNR', 'LR', 'RG', 'LSVR', 'DTR', 'RFR',
    'GBR']

## Analisando resultado pelo erro

**Tipos de erro**

In [ ]:
def calculate_mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))
def calculate_rmse(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.sqrt(np.mean(pow((y_true - y_pred),2)))

In [ ]:
erro_data = []
product = None
data_produto = final.copy()
data_produto['data'] = pd.to_datetime(data_produto['year'].astype(str) + '-' + data_produto['month'].astype(str) + '-01')
erro_data = [
    ["MAPE", *[str(calculate_mape(data_produto[real_value], data_produto[algo])).replace(".", ",") for algo in columns_with_results]],
    ["RMSE", *[str(calculate_rmse(data_produto[real_value], data_produto[algo])).replace(".", ",") for algo in columns_with_results]]
]
erro_df = pd.DataFrame(
    erro_data,
    columns=["Metodo", "Media Movel 3","Media Movel 5","Media Movel 7", "Media Movel Ponderada 2","Media Movel Ponderada 3","Media Movel Ponderada 4","ARIMA",	"KNR",	"LR",	"RG",	"LSVR",	"DTR",	"RFR",	"GBR"],

)
erro_df.to_excel("erro_total_result.xlsx")